In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [4]:
df = pd.read_excel('students.xlsx')
df

,ASS. NO:,NAME,ENG,KISW,MATH,INT SCI,AGRI,SST,CRE,CAS,PTE,TT POINTS
0,B001880939,GEORGE MIGAEL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A001050410,SAMUEL MBUGUA GITHINJI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A001409207,BRIGIT WAMBUI GICHIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A001220772,DAVIS BARAKA SUMBA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,B001986866,FAITH KWAMBOKA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
479,A000133761,ALPHA NGAIRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
480,B001987554,JEREMIA MUSYOKA MUENI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
481,B001986798,ELIZABETH MUTHEU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
482,A000898909,MONICA NDUKU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
driver = webdriver.Firefox()
wait = WebDriverWait(driver, 20)

URL = "https://kjsea.knec.ac.ke/"

In [6]:
row = df.iloc[0]

driver.get(URL)

ass_input = wait.until(
    EC.presence_of_element_located(
        (By.XPATH, "//input[@placeholder='Assessment number (9 or 10 characters) *']"))
)

name_input = driver.find_element(
    By.XPATH, "//input[@placeholder='Atleast one name of the candidate *']"
)

ass_input.clear()
ass_input.send_keys(str(row['ASS. NO:']))

name_input.clear()
name_input.send_keys(str(row['NAME']))

checkbox = driver.find_element(
    By.XPATH, "//input[@type='checkbox']"
)
checkbox.click()


search = driver.find_element(
    By.XPATH, "//button[contains(., 'Search Results')]"
)
try:
    search_btn = wait.until(
        EC.element_to_be_clickable(search)
    )

    time.sleep(1) 
    

    search_btn.click()
    print("Attempted to click 'Search Results' button using standard click.")

    time.sleep(5) 
    
except Exception as e:
    print(f"Error during search execution: {e}")

Attempted to click 'Search Results' button using standard click.


In [7]:
# fetching all subjects 
def get_points(subject_name):
    return driver.find_element(
        By.XPATH, f"//tr[td[normalize-space()='{subject_name}']]/td[4]"
    ).text

In [8]:
test_points = {
    "ENG" : get_points("English"),
    "KISW": get_points("Kiswahili"),
    "MATH": get_points("Mathematics"),
    "INTSC": get_points("Integrated Science"),
    "AGRI": get_points("Agriculture"),
    "SST":get_points("Social Studies"),
    "CRE":get_points("Christian Religious Education"),
    "CAS":get_points("Creative Arts & Sports"),
    "PTE":get_points("Pre-technical Studies")
}

In [9]:
test_points

{'ENG': '7',
 'KISW': '7',
 'MATH': '7',
 'INTSC': '8',
 'AGRI': '8',
 'SST': '8',
 'CRE': '8',
 'CAS': '7',
 'PTE': '7'}

In [10]:
for index, row in df.iterrows():
    driver.get(URL)
    try:
        print(f"Fetching results for {row['NAME']} ({row['ASS. NO:']})...")
        ass_input = wait.until(
            EC.presence_of_element_located(
                (By.XPATH, "//input[@placeholder='Assessment number (9 or 10 characters) *']"))
                 )

        name_input = driver.find_element(
             By.XPATH, "//input[@placeholder='Atleast one name of the candidate *']"
             )

        ass_input.clear()
        ass_input.send_keys(str(row['ASS. NO:']))

        name_input.clear()
        name_input.send_keys(str(row['NAME']))

        checkbox = driver.find_element(By.XPATH, "//input[@type='checkbox']")
        checkbox.click()

        search_button = driver.find_element(By.XPATH, "//button[contains(., 'Search Results')]")
           # Using JavaScript click to avoid interception issues\n",
        driver.execute_script("arguments[0].click();", search_button)
           
           # Wait for results to load by checking for a known element\n",
        wait.until(EC.presence_of_element_located((By.XPATH, f"//tr[td[normalize-space()='English']]")))
           
           # --- Data Extraction (moved inside the loop) ---\n",
        df.at[index, 'ENG'] = get_points("English")
        df.at[index, 'KISW'] = get_points("Kiswahili")
        df.at[index, 'MATH'] = get_points("Mathematics")
        df.at[index, 'INT SCI'] = get_points("Integrated Science")
        df.at[index, 'AGRI'] = get_points("Agriculture")
        df.at[index, 'SST'] = get_points("Social Studies")
        df.at[index, 'CRE'] = get_points("Christian Religious Education")
        df.at[index, 'CAS'] = get_points("Creative Arts & Sports")
        df.at[index, 'PTE'] = get_points("Pre-technical Studies")
        print("Successfully extracted points.")

    except Exception as e:
           print(f"Could not fetch results for {row['NAME']}. Error: {e}")
           continue

Fetching results for GEORGE MIGAEL (B001880939)...


/tmp/ipykernel_34376/2910777152.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '7' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'ENG'] = get_points("English")
/tmp/ipykernel_34376/2910777152.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '7' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'KISW'] = get_points("Kiswahili")
/tmp/ipykernel_34376/2910777152.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '7' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'MATH'] = get_points("Mathematics")
/tmp/ipykernel_34376/2910777152.py:34: FutureWarning: Setting an item of i

Successfully extracted points.
Fetching results for SAMUEL MBUGUA GITHINJI (A001050410)...
Successfully extracted points.
Fetching results for BRIGIT WAMBUI GICHIA (A001409207)...
Successfully extracted points.
Fetching results for DAVIS BARAKA SUMBA (A001220772)...
Successfully extracted points.
Fetching results for FAITH KWAMBOKA (B001986866)...
Successfully extracted points.
Fetching results for MERCY MWENDE KYALO (B001988120)...
Successfully extracted points.
Fetching results for CLIFFORD OBAMA (A002360178)...
Successfully extracted points.
Fetching results for RUTH MUKUI KIUNGUA (A000959924)...
Successfully extracted points.
Fetching results for BLESSING KABURU (B002835517)...
Successfully extracted points.
Fetching results for SYLVIA ENGEFU LIVANZE (A000476471)...
Successfully extracted points.
Fetching results for JOSPHAT MWONGELA (B002835699)...
Successfully extracted points.
Fetching results for SUSAN MWENDE MOKI (A000405767)...
Successfully extracted points.
Fetching results 

In [11]:
df.to_excel('students_results.xlsx', index=False)
driver.quit()